Github: https://github.com/05satyam/AI-ML/tree/main


# **Some key terminologies:**
- LLM (Large Language Model): AI trained on vast datasets to understand and generate human-like text.
- RAG (Retrieval-Augmented Generation): Combines LLMs with retrieval systems for context-aware responses.
- Embeddings: Numerical representation of data for semantic similarity in vector space.
- Fine-Tuning: Customizing a pre-trained model for specific tasks or domains.
- Prompt Engineering: Crafting inputs to elicit desired outputs from LLMs.
- Guardrails: Safety mechanisms to ensure reliable, ethical model outputs.
- Vector Database: Specialized database for storing and querying embeddings.
- Agents: Autonomous units in AI systems capable of decision-making and task execution.

- Below code from documentations of companies like Langchain, Pydantic, Crew, LlamaIndex.

In [3]:
!pip install llama-index llama-index-callbacks-arize-phoenix

In [4]:
!pip install langchain-openai
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


## NAIVE-RAG

In [8]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')


### Load embedding models, we are using OpenAI Embeddings for easy access!!

In [6]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")

### ETI -> extract--transform--ingest

- Load the data soruce, we can load multiple documents, here we are using attention is all you need pdf

In [10]:
# load data
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()

### Text splitter with a default chunk size of 1000, overlap 200
It is always best to experiment with these more.

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszk

#### Different types of chunking

In [9]:
# Fixed-Size Chunking: Splits text into equal-sized chunks (with optional overlap).

from langchain.text_splitter import CharacterTextSplitter

text = "Photosynthesis is one of nature's most vital processes. It allows plants to convert sunlight into energy."

splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=10)
chunks = splitter.split_text(text)

print("Fixed size....")
print(chunks)

print()

# 2 Recursive Chunking: Splits text first by paragraphs, then by sentences if needed.
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = """Quantum entanglement is a key concept in quantum physics.
It occurs when particles become linked, so the state of one affects the state of another."""

splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
chunks = splitter.split_text(text)

print("Recursive Chunking:")
print(chunks)

print()


# 3 Document-Based Chunking: Splits text based on structured elements (like Markdown headers).
from langchain.text_splitter import MarkdownTextSplitter

text = """# Heading
This is a heading.

## Subheading
This is a subheading. We can continue with more content."""

splitter = MarkdownTextSplitter()
chunks = splitter.split_text(text)
print("Document Based")
print(chunks)
print()


# 4. Semantic Chunking: Splits text into chunks based on meaning (using embeddings). : Preserves semantic coherence.
import numpy as np
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=userdata.get('OPENAI_API_KEY'))

# Sample text
text = """The water cycle is a continuous process by which water moves through the Earth's atmosphere.
It involves evaporation, condensation, precipitation, and collection.
Evaporation occurs when heat turns water into vapor.
This vapor rises, cools, and forms clouds.
Eventually, precipitation falls as rain, snow, or sleet, continuing the cycle."""

# Split text into sentences first
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
sentences = text_splitter.split_text(text)

# Generate embeddings for each sentence
sentence_vectors = np.array([embeddings.embed_query(sentence) for sentence in sentences])

# Function to calculate cosine similarity
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Merge sentences based on semantic similarity threshold
chunks = []
current_chunk = [sentences[0]]

for i in range(1, len(sentences)):
    sim = cosine_similarity(sentence_vectors[i-1], sentence_vectors[i])

    # If similarity drops below a threshold, start a new chunk
    if sim < 0.7:
        chunks.append(" ".join(current_chunk))
        current_chunk = [sentences[i]]
    else:
        current_chunk.append(sentences[i])

# Append the last chunk
if current_chunk:
    chunks.append(" ".join(current_chunk))

# Print the semantically grouped chunks
print("Semantic Chunking:")
print(chunks)
print()

# 5 LLM-Based Chunking: Uses an LLM to generate chunks based on meaning.
import os
from langchain.llms import OpenAI

# Initialize OpenAI LLM
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

# Sample text
text = """AI is transforming the world. It is used in healthcare, finance, and various industries.
Models like GPT understand language better than ever.
However, AI also presents challenges in terms of ethics and bias.
Regulation and careful development are needed to ensure AI benefits humanity."""

# **LLM Prompt for Chunking**
prompt = f"""
You are an AI assistant helping to split text into meaningful chunks.
Given the following text, suggest where to break it into separate logical chunks:
{text}
Provide the output as a list of chunked text blocks.
"""

# Get response from OpenAI
response = llm.predict(prompt)

# Print LLM-suggested chunks
print("LLM based chunking")
print(response)

print()

Fixed size....
["Photosynthesis is one of nature's most vital processes. It allows plants to convert sunlight into energy."]

Recursive Chunking:
['Quantum entanglement is a key concept in quantum', 'quantum physics.', 'It occurs when particles become linked, so the', 'so the state of one affects the state of another.']

Document Based
['# Heading\nThis is a heading.\n\n## Subheading\nThis is a subheading. We can continue with more content.']



<ipython-input-9-282e372ab84a>:52: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=userdata.get('OPENAI_API_KEY'))


Semantic Chunking:
["The water cycle is a continuous process by which water moves through the Earth's atmosphere. It involves evaporation, condensation, precipitation, and collection. Evaporation occurs when heat turns water into vapor. \nThis vapor rises, cools, and forms clouds. Eventually, precipitation falls as rain, snow, or sleet, continuing the cycle."]



<ipython-input-9-282e372ab84a>:100: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
<ipython-input-9-282e372ab84a>:117: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


LLM based chunking

1. "AI is transforming the world. It is used in healthcare, finance, and various industries."
2. "Models like GPT understand language better than ever."
3. "However, AI also presents challenges in terms of ethics and bias."
4. "Regulation and careful development are needed to ensure AI benefits humanity."



In [12]:
!pip install chromadb # hnsw, index count, embedding algorithms dim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00

### Data Ingestion

In [13]:
#Chroma DB - Opensource, local db setup

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings())

In [15]:
query = "Who are the authors of attention is all you need?"
retireved_results=db.similarity_search(query)


In [16]:
print(retireved_results[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions


In [18]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [19]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

RetrievalQA is a specialized LangChain chain that retrieves relevant documents from a vector store and feeds them into an LLM to answer user queries.

In [21]:
# Define simple RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY")),
    chain_type="stuff",
    retriever=retriever
)

In [22]:
query = "Who are the authors of 'Attention Is All You Need'?"
response = rag_chain.run(query)

print("Bot:", response)

<ipython-input-22-12978c8f6f17>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)


Bot:  Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin


### Advance RAg

In [26]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# **Fix**: Define memory with explicit `output_key`
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"  # Explicitly set the output key to "answer"
)

# Define advanced RAG chain with fixed memory settings
advanced_rag_chain = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY")),
    retriever=retriever,
    memory=memory,
    return_source_documents=True  # Includes document sources
)

# Simulate chatbot conversation
chat_history = []
query = "can you talk more about encoders"
# query transformation by llm :
response = advanced_rag_chain.invoke({"question": query, "chat_history": chat_history})

# Extract answer and source documents
answer = response.get("answer", "No answer found.")
sources = response.get("source_documents", [])

# Display results
print("Bot:", answer)
if sources:
    print(f"\nSources: {sources}")


Bot:  Yes, encoders are a key component in many neural sequence transduction models. In this context, the encoder maps an input sequence of symbols to a sequence of continuous representations. This allows for efficient processing and generation of output sequences by the decoder. The Transformer is a notable model that relies entirely on self-attention for encoding, without using sequential RNNs or convolution. 

Sources: [Document(metadata={'author': '', 'creationdate': '2023-08-03T00:07:29+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'page': 1, 'page_label': '2', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'attention.pdf', 'subject': '', 'title': '', 'total_pages': 15, 'trapped': '/False'}, page_content='Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence\nof continuous representations 

# Agents

## Langchain based

In [28]:
!pip install langchain langchain_openai langchain_community google-search-results
!pip install -U duckduckgo-search


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=b8cbcaa1e887b1e178a47f55276199a87623bdcda347ae4d728f10bb4473a29e
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.4 MB/s eta 0:00:00


- This code creates a simple AI agent that can:

    - Search the web using DuckDuckGo (no API key needed)
    - Perform calculations using a built-in calculator tool
    - Decide dynamically which tool to use based on the query

### Initialize LLM
2️### Define Tools
  🔍 DuckDuckGo Search Tool
  🧮 Calculator Tool

### Initialize the Agent
   - Uses both tools (search_tool & calculator).
   - AgentType.ZERO_SHOT_REACT_DESCRIPTION → Enables reasoning before answering.
   - verbose=True → Prints step-by-step decisions.
  
### Run Queries


### AgentTypes:

  1. Zero-Shot ReAct (AgentType.ZERO_SHOT_REACT_DESCRIPTION)
    - Best for: Simple, single-step reasoning tasks
    - How it works:

    - Uses the ReAct (Reasoning + Acting) framework.
    - The model reads the query, decides which single tool to use, and executes.
    - No intermediate steps—just one reasoning step before acting.


2. Conversational Agent (AgentType. CONVERSATIONAL_REACT_DESCRIPTION)
    - Best for: Multi-turn conversations with memory
    - How it works:

    - Similar to Zero-Shot ReAct but with memory support.
    - Keeps track of past interactions to provide context-aware responses.
    - agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,

- **And many more..**

- Choosing the Right Agent
    - For simple Q&A → ZERO_SHOT_REACT_DESCRIPTION
    - For chatbots with memory → CONVERSATIONAL_REACT_DESCRIPTION
    - For API interaction → STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
    - For function-calling LLMs → OPENAI_FUNCTIONS
    - For deep research tasks → SELF_ASK_WITH_SEARCH

In [29]:
import os
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

# Define Search Tool (DuckDuckGo - No API key required)
search_tool = DuckDuckGoSearchRun()

# Define Calculator Tool
def calculator_tool(expression):
    """Evaluate simple math expressions."""
    try:
        return eval(expression)
    except Exception as e:
        return str(e)

calculator = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Use this tool to solve mathematical expressions."
)

# Create Agent with Tools
agent = initialize_agent(
    tools=[search_tool, calculator],  # Define agent tools
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Zero-shot agent
    verbose=True  # Show step-by-step execution
)

# Example Queries
queries = [
    "Who is the CEO of OpenAI?",
    "What is 2345 * 789?"
]

for query in queries:
    print(f"\nUser: {query}")
    response = agent.run(query)
    print(f"Agent: {response}")


<ipython-input-29-37d93c79b4d8>:28: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(



User: Who is the CEO of OpenAI?


> Entering new AgentExecutor chain...
 I should use DuckDuckGo Search to find the answer
Action: duckduckgo_search
Action Input: "CEO of OpenAI"
Observation: OpenAI CEO Sam Altman rejected a potentially seismic deal that could shape the future of artificial intelligence, telling the Elon Musk-led group of investors that the ChatGPT maker is not for sale. OpenAI CEO Sam Altman had an eventful 2024, and 2025 is shaping up to be just as big. While the 39-year-old entrepreneur has been a household name in Silicon Valley for years now, the rest of the ... Before he became the CEO, Altman was the co-chair of OpenAI when it was founded in 2015, along with Tesla CEO Elon Musk. While OpenAI initially started as a nonprofit research company with a focus ... Sam Altman is the CEO of OpenAI and a prolific venture investor. In 2005, he dropped out of Stanford to found social mapping company Loopt, which sold in 2012 for $43 million; he used proceeds to ... Sam Alt

## Crew AI based Single AGENT Application

In [30]:
#Crewai documentaion reference's
# !pip install crewai
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You mi

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
from crewai import Agent, Task, Crew

In [6]:
import os
# from utils import get_openai_api_key

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4'

In [9]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

Agent Writer

In [10]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

Agent Editor

In [11]:
#Edit task
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

Creating task

In [12]:
#plan Task
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [13]:
#Write Task
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [14]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

Create your crew of Agents
Pass the tasks to be performed by those agents.
Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
verbose=2 allows you to see all the logs of the execution.

In [15]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [16]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.

Latest trends:
   - AI in Healthcare: The use of AI in predicting diseases, drug discovery, radiology, etc.
   - AI in Cybersecurity: AI's crucial role in detecting and preventing cyber attacks.
   - AI in Retail: Predictive analytics, personalized shopping, and inventory management.
   - AI in Autonomous Vehicles: The advancement of self-driving technology.

Key Players:
   - Google: DeepMind, Google AI
   - Micros

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Intricacies of Artificial Intelligence: An In-Depth Overview 

Artificial Intelligence (AI) has emerged as a game-changing technology, revolutionizing various facets of our lives, from healthcare to customer service. As a confluence of several sub-disciplines, AI holds the potential to amplify human abilities, automate routine tasks, and open up fresh avenues in scientific exploration. 

## Groundbreaking Trends in AI

AI is not a static realm; it's a whirlwind of innovation and pioneering trends. Machine Learning, a subset of AI, employs statistical methods to empower machines to enhance with experience. This key trend is propelling advancements in predictive analytics and data mining. Reinforcement Learning, another burgeoning trend, employs reward-based systems to train models, powering applications like autonomous cars and game-playing bots. 

AI's role in healthcare is a noteworthy trend. From predicting diseases to tailoring treatments, AI is turning into a healthcare game-changer. AI is also redefining customer service, with AI-powered chatbots offering 24/7 support. Amid these trends, concerns around AI ethics, such as data privacy and algorithmic bias, have become prominent.

## The Majors Players in AI

The swift rise of AI can be attributed to key players who are pushing the boundaries of what's feasible. Tech titans like Google, Microsoft, IBM, Amazon, and Facebook are at the forefront, investing heavily in AI research and development. Google's DeepMind made headlines with its AI AlphaGo, which triumphed over human players in the complex game of Go. 

Microsoft's AI initiatives are revolutionizing healthcare, employing AI to predict patient readmissions and personalize care. Amazon's AI powers Alexa, its virtual assistant, while Facebook's deep learning research is enhancing AI capabilities. Not to be overlooked, OpenAI's GPT-3 has set a new bar in natural language processing.

## Recent AI Chronicles: Notable News and Developments

The AI landscape is in constant flux, with new developments reshaping the field. Facebook's advancements in deep learning, Microsoft's applications of AI in healthcare, Google's victory in Go, and Amazon's Alexa are notable news that underline AI's potential. OpenAI's GPT-3, a language prediction model, has also made ripples, heralding a new era in AI capabilities. 

## Demystifying AI: Addressing Complexities and Staying Current

Understanding AI can be challenging, given its breadth and complexity. However, resources like arXiv's research papers and tech blogs provide insights into the intricate world of AI. Staying updated is vital, with platforms like TechCrunch and The Verge offering the latest AI news.

## Looking Forward: AI Predictions and Ethical Considerations

The future of AI looks bright, with predictions of AI becoming increasingly ingrained in our everyday lives. However, along with these advancements, we must address AI ethics. Concerns around data privacy, algorithmic bias, and job displacement are valid and require consideration as we shape AI's future.

In conclusion, AI is a dynamic, transformative technology that's reshaping our world. Both its advancements and ethical implications warrant our attention and engagement. So, let's stay informed and actively participate in the AI dialogue. After all, AI is not just the future; it's the present.

In [ ]:
'''
Try with another llm
Hugging Face (HuggingFaceHub endpoint)
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()
'''

'\nTry with another llm\nHugging Face (HuggingFaceHub endpoint)\nfrom langchain_community.llms import HuggingFaceHub\n\nllm = HuggingFaceHub(\n    repo_id="HuggingFaceH4/zephyr-7b-beta",\n    huggingfacehub_api_token="<HF_TOKEN_HERE>",\n    task="text-generation",\n)\n\n\n'

## Multi-agent

 Agent can delegate its work to another agent which is better suited to do a particular task.


 allow_delegation=False,

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

Tools

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

**Different Ways to Give Agents Tools**
1. Agent Level: The Agent can use the Tool(s) on any Task it performs.
2. Task Level: The Agent will only use the Tool(s) when performing that specific Task.

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
The customer needs guidance on how to set up a Crew and specifically how to add memory to the Crew. I need to ensure that I provide a full answer to their question. Since I don't have this information readily available, I will need to use the "Read website content" tool to read the relevant content from the crewAI documentation.

Action: 
Read website content

Action Input: 
{
  "url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
}
 

Int

In [ ]:
from IPython.display import Markdown
Markdown(result)

Dear Andrew,

Thank you for reaching out to us. CrewAI is a powerful platform that allows you to create a group of AI agents, each with specific roles, tools, and goals to accomplish complex tasks. This group is what we call a "Crew". 

You've asked about how to add memory to a Crew. I'm sorry for the oversight in my previous response. Let me provide you with the specific steps you can follow:

1. Navigate to the Crews page in your CrewAI dashboard.
2. Click on the specific Crew you want to add memory to.
3. On the Crew details page, find the "Memory" section.
4. Click on the "Add Memory" button.
5. Input the amount of memory you want to add.
6. Confirm your changes by clicking on the "Update" button.

These instructions are also available in our documentation. Please go to the "Managing Crews" section, and look for the subsection titled "Adding Memory to a Crew". 

If you need further assistance or have any other queries, please don't hesitate to ask. We're here to help!

Best regards,
[Your Name]
Senior Support Representative, crewAI

In [ ]:
!pip install llama-index llama-index-callbacks-arize-phoenix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/9

LLM-Observability

In [ ]:
import os
import llama_index.core
from llama_index.core import set_global_handler



PHOENIX_API_KEY = "87a00ca31b5789s696"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)


In [ ]:
from llama_index.llms.openai import OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [ ]:
from llama_index.core import VectorStoreIndex, Settings, Document

import openai

llm = OpenAI(model="gpt-4o", temperature=0)

Settings.llm = llm

In [ ]:
documents = [
    Document(text="One Piece does exist!! ~ Whitebeard"),
    Document(text="LlamaIndex helps you build structured applications on top of LLMs."),
    Document(text="Arize Phoenix provides observability for machine learning and LLM applications.")
]
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
question = "one piece"
response = query_engine.query(question)
print(response)

One Piece is confirmed to exist by Whitebeard.
